In [1]:
# 端到端的多示例学习,从MRI图像+实例分割掩膜中提取特征,预测病人类型.

In [2]:
## import 
import argparse
from collections import Counter
import copy
from glob import glob

import json
import matplotlib.pyplot as plt
import math
import numpy as np

import os
import pandas as pd
import pdb
import random

import scipy
import SimpleITK as sitk

from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.model_selection import StratifiedKFold

import torch
from torch.utils.data import DataLoader, sampler, Dataset
import torch.nn as nn
import torch.nn.functional as F
import time

from models.Onlymask_resnet import Resmode
from utils import *

In [3]:
# Training settings
parser = argparse.ArgumentParser(description='Configurations for Mission1 Training')

parser.add_argument('--max_epoch',
                    type=int,
                    default=100,
                    help='maximum number of epochs to train (default: 200)')
parser.add_argument('--lr',
                    type=float,
                    default=1e-4,
                    help='learning rate (default: 0.0001)')
parser.add_argument('--fold',
                    type=int,
                    default=10,
                    help='number of folds (default: 1)')
parser.add_argument('--results_dir',
                    default='./results',
                    help='results directory (default: ./results)')

parser.add_argument('--log_data',
                    action='store_true',
                    default=False,
                    help='log data using tensorboard')
parser.add_argument('--opt', type=str, choices=['adam', 'sgd'], default='adam')
parser.add_argument('--drop_out',
                    action='store_true',
                    default=False,
                    help='enabel dropout (p=0.25)')
parser.add_argument('--backbone_requires_grad',
                    action='store_true',
                    default=True,
                    help='whether to train the backbone')
parser.add_argument('--input_channel',
                    type=int,
                    default=5,
                    help='number of input image channels')
parser.add_argument('--gpu', type=str, default='1', help='GPU to use')
args = parser.parse_args(args=[])  # args=[]
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_classes = 2
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

In [4]:
# Dataset
class M1_dataset(Dataset):
    def __init__(self, path_list, label_list, mode, aug=0.3):
        self.All_data = path_list
        self.All_label = label_list
        self.mode = mode
        self.aug = aug
        self.phase = ['_T1', '_T2', '_AP', '_PP', '_20 MIN']

        self.index_0 = [x for x, y in list(enumerate(label_list)) if y == 0]
        self.index_1 = [x for x, y in list(enumerate(label_list)) if y == 1]
        

    def __getitem__(self, index):
           # transformer
        def random_drop(img_clip, max_rate=0.2):
            c = img_clip.shape[1]
            d = img_clip.shape[2]
            # 随机生成一个立方体
            cut_c = np.random.rand(1)
            cut_d = np.random.rand(1)

            while cut_c*cut_d > max_rate:
                if np.random.rand(1) > 0.5:
                    cut_c = max(cut_c*0.8, 0.01)
                else:
                    cut_d = max(cut_d*0.8, 0.01)

            drop_d1 = np.random.randint(0, d-d*cut_d+1)
            drop_c1 = np.random.randint(0, c-c*cut_c+1)

#             if np.random.rand(1)>0.5:
            img_clip[:,int(drop_c1):int(min(drop_c1+c*cut_c, c)),
                     int(drop_d1):int(min(drop_d1+d*cut_d, d))] = np.zeros(( 25,
                                                                           (int(min(drop_c1+c*cut_c, c)-drop_c1)),
                                                                           (int(min(drop_d1+d*cut_d, d)-drop_d1))))
#             else:
#                 img_clip[:,int(drop_c1):int(min(drop_c1+c*cut_c, c)),
#                          int(drop_d1):int(min(drop_d1+d*cut_d, d))] = np.ones(( 5,
#                                                                                (int(min(drop_c1+c*cut_c, c)-drop_c1)),
#                                                                                (int(min(drop_d1+d*cut_d, d)-drop_d1))))
            return img_clip

        def random_clip(img_clip, max_rate=0.1):
            a = img_clip.shape[0]
            c = img_clip.shape[2]
            d = img_clip.shape[3]
            drop_rate1 = np.random.randint(0, max_rate*100) / 100
            drop_rate2 = np.random.randint(0, max_rate*100) / 100

            if (np.random.rand(1) > 0.5) and (a > 10):
                img_clip = img_clip[int(a * drop_rate1):
                                    int(a * (1 - drop_rate2)), :, :, :]
            if (np.random.rand(1) > 0.5) and (c > 10):
                img_clip = img_clip[:, :, int(c * drop_rate1):
                                    int(c * (1 - drop_rate2)), :]
            if (np.random.rand(1) > 0.5) and (d > 10):
                img_clip = img_clip[:, :, :, int(d * drop_rate1):
                                    int(d * (1 - drop_rate2))]
            return img_clip

        def random_flip(img_clip, flip_rate=0.5):
            if np.random.rand(1) > flip_rate:
                img_clip = img_clip[:, ::-1, :].copy()
            if np.random.rand(1) > flip_rate:
                img_clip = img_clip[:, :, ::-1].copy()
            return img_clip

        def resize_np(img, img_new):
            a = img.shape[0]
            c = img.shape[1]
            a_new = img_new.shape[0]
            c_new = img_new.shape[1]
            img = scipy.ndimage.zoom(
                img, (a_new/a,  c_new/c))
            return img

        def copy_channel(img, channel=0):
            img_clip = np.concatenate(
                [img_clip.copy(), img_clip.copy()], axis=channel)

        def mix_up(img, target):
            if target == 0:
                index_supply = random.choice(self.index_0)
            else:
                index_supply = random.choice(self.index_1)
            name_file_supply = self.All_data[index_supply]

            # 读取新的数据
            supply_img = np.zeros((25, 32, 32))
            n = 0

            for phase_id in range(5):
                channel_name = name_file_supply[:-4] + self.phase[phase_id] + '.npy'
                img_channel = np.load(channel_name)

                img1 = np.max(img_channel, axis=0)
                img2 = np.mean(img_channel, axis=0)
                img3 = np.min(img_channel, axis=0)
                img4 = img1-img3
                img5 = np.median(img_channel, axis=0)

                for img_fill in [img1, img2, img3, img4, img5]:
                    img_fill = img_fill/(img1.max()+0.01)
                    img_fill = resize_np(img_fill, np.zeros((32, 32)))
                    supply_img[n, :, :] = img_fill
                    n+=1
            
            ratio = np.random.rand(1)/2+0.5
            img = img*ratio + supply_img*(1-ratio)
            return img


        name_file, target = self.All_data[index], self.All_label[index]
        output_img= np.zeros((25, 32, 32))
        n = 0
        
        for phase_id in range(5):
            channel_name = name_file[:-4] + self.phase[phase_id] + '.npy'
            img_channel = np.load(channel_name)
            
            img1 = np.max(img_channel, axis=0)
            img2 = np.mean(img_channel, axis=0)
            img3 = np.min(img_channel, axis=0)
            img4 = img1-img3
            img5 = np.median(img_channel, axis=0)

            for img_fill in [img1, img2, img3, img4, img5]:
                img_fill = img_fill/(img1.max()+0.01)
                img_fill = resize_np(img_fill, np.zeros((32, 32)))
                output_img[n, :, :] = img_fill
                n+=1

        if self.mode =='train':
            # 随机翻转
            if np.random.rand(1)<1/3:
                output_img = mix_up(output_img, target=target)
            elif np.random.rand(1)<1/2:
                output_img = random_drop(output_img, max_rate=self.aug)
            output_img = random_flip(output_img, flip_rate=0.5)


        return output_img, target, name_file[:-4]

    def __len__(self):
        return len(self.All_data)

In [5]:
# 获取样本比例
def get_weights(train_label_list):
    Count_sample = dict(Counter(train_label_list))
    N = float(len(train_label_list))
    weight_per_class = [1 / Count_sample[0], 1 / Count_sample[1]]

    weight = [0] * int(N)
    for idx in range(len(train_label_list)):
        y = train_label_list[idx]
        weight[idx] = weight_per_class[y]
    weights = torch.DoubleTensor(weight)
    return weights


def train_simple(epoch, net, optimizer, dataloader):
    num_iter = (len(dataloader.dataset) // dataloader.batch_size) + 1

    for batch_idx, (inputs, labels, _) in enumerate(dataloader):
        net.train()
        inputs, labels = inputs.to(torch.float32).cuda(), labels.cuda()
        optimizer.zero_grad()   # reset gradient
        outputs, _ = net(inputs)
         
        L = F.cross_entropy(outputs, labels)
        L.backward()
        optimizer.step()
                
        if ((batch_idx + 1) % 10 == 0) or (batch_idx + 1  == num_iter):
            print('| Epoch [%3d/%3d] Iter[%3d/%3d]\t CE-loss: %.4f' %
                  (epoch, args.max_epoch, batch_idx + 1, num_iter, L.item()))
            


def Net_val(epoch, net, dataloader):
    net.eval()

    correct = 0
    total = 0
    loss_x = 0
    n = 0
    
    y_true = np.zeros(len(dataloader.dataset))
    y_hat = np.zeros(len(dataloader.dataset))
    y_score = np.zeros((len(dataloader.dataset),2))
    
    with torch.no_grad():
        for _, (inputs, targets, _) in enumerate(dataloader):

            inputs = inputs.to(torch.float32).cuda()
            outputs, _ = net(inputs)

            predicted = torch.argmax(outputs, -1) 
            loss = F.cross_entropy(outputs, targets.cuda())
            loss_x += loss.item()
            
            for b in range(targets.size(0)):
                y_true[n] = targets[b]
                y_hat[n] = predicted[b]
                y_score[n,:] = np.array(outputs.cpu())[b,:]
                n += 1       
            
            total += targets.size(0)
            correct += predicted.eq(targets.cuda()).cpu().sum().item()

    acc = 100. * correct / total
    loss_x = loss_x/total
            
    # 计算auc        
    roc_auc = metrics.roc_auc_score(y_true, y_score[:,1])
    spe_auc = metrics.roc_auc_score(1-y_true, y_score[:,0])
    
    print("\n| Test Epoch #%d\t Accuracy: %.2f%%\t Loss: %.2f\t AUC: %.3f\t SPE: %.3f\n" %
                  (epoch, acc, loss_x, roc_auc, spe_auc))
    
    return (acc, loss_x, roc_auc, spe_auc), (y_true, y_hat, y_score)


def bootstrap_score(func, y, pred, classes, bootstraps = 100, fold_size = 1000):
    '''
    func: callable score function
    y不是onehot
    '''
    statistics = np.zeros((len(classes), bootstraps))

    for k,c in enumerate(classes):
        df = pd.DataFrame(columns=['y', 'pred'])
        df.loc[:, 'y'] = y
        try:
            df.loc[:, 'pred'] = pred[:, -1]
        except:
            df.loc[:, 'pred'] = pred[:]

        df_pos = df[df.y == 1]
        df_neg = df[df.y == 0]
        prevalence = len(df_pos) / len(df)
        for i in range(bootstraps):
            pos_sample = df_pos.sample(n = int(fold_size * prevalence), replace=True)
            neg_sample = df_neg.sample(n = int(fold_size * (1-prevalence)), replace=True)

            y_sample = np.concatenate([pos_sample.y.values, neg_sample.y.values])
            pred_sample = np.concatenate([pos_sample.pred.values, neg_sample.pred.values])
            score = func(y_sample, pred_sample)
            statistics[k][i] = score
    return statistics

In [6]:
## # 分层K折交叉验证
filepath_df = pd.read_csv('../All_keep_v3/label_change6.csv')
filepath_list = filepath_df.path
label_list = filepath_df.label
aug = 0.6
ACC_score = [91.566, 89.157, 87.952, 84.337, 86.747,
            83.133, 84.337, 85.542, 91.566, 86.585]
AUC_score = [0.943, 0.939, 0.934, 0.922, 0.923,
            0.869, 0.893, 0.925, 0.955, 0.918]
LOSS_score = [1.662, 2.179, 2.077, 2.372, 2.462,
             4.040, 2.544, 2.661, 2.215, 2.124]

history = {'train_acc':[],
           'test_acc':[],
           'train_loss':[],
           'test_loss':[]}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i_fold, (train_index,
        test_index) in enumerate(skf.split(X=filepath_list, y=label_list)):
    if i_fold >= 0:
        # 数据集划分
        # X:
        train_filepath_list = [filepath_list[idx] for idx in train_index]
        test_filepath_list = [filepath_list[idx] for idx in test_index]
        # y:
        train_label_list = [label_list[idx] for idx in train_index]
        test_label_list = [label_list[idx] for idx in test_index]

        # dataset object
        train_dataset = M1_dataset(path_list=train_filepath_list,
                                   label_list=train_label_list, mode='train', aug=aug)
        test_dataset = M1_dataset(path_list=test_filepath_list,
                                  label_list=test_label_list, mode='test')
        train_eval_dataset = M1_dataset(path_list=train_filepath_list,
                               label_list=train_label_list, mode='eval')

        # dataloader
        train_eval_loader = DataLoader(train_eval_dataset, batch_size=16, shuffle=False)
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

        print('Building dataloader: training: {}, testing: {}\n'.format(len(train_dataset), len(test_dataset)))

        # model initialize
        model = Resmode(input_channel=5).to(args.device)
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, weight_decay=0.0001)
        best_acc = 0
        best_auc = 0
        best_loss = 5

        for epoch in range(args.max_epoch):
            lr = 0.0001 * (0.1 ** int(epoch >= 15)) * (0.1 ** int(epoch >= 40))
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

            train_simple(epoch, model, optimizer, train_loader)

            print('Now evaluating on fold {}'.format(i_fold))
            (acc_train, loss_train, auc_train, spe_train), (train_true, train_hat, train_score) = Net_val(epoch, model, train_eval_loader)

            (acc_test, loss_test, auc_test, spe_test), (test_true, test_hat, test_score) = Net_val(epoch, model, test_loader)

            history['train_acc'].append(acc_train)
            history['test_acc'].append(acc_test)
            history['train_loss'].append(loss_train)
            history['test_loss'].append(loss_test)
            print(model.get_gates())
            
    #         if auc_train >= auc_test:                                
            if best_acc <= acc_test:
                best_acc = acc_test
                if best_acc > 70:
                    torch.save(model.state_dict(), '../log4/model_OMv7_fold{}_acc0208.pth'.format(i_fold))
                    print('saving model')

            if best_auc <= auc_test:
                best_auc = auc_test
                if best_auc > 0.8:
                    torch.save(model.state_dict(), '../log4/model_OMv7_fold{}_auc0208.pth'.format(i_fold))
                    best_gate = F.softmax(model.get_gates(), dim=-1)
                    best_gate = np.array(best_gate.detach().cpu())
                    print('saving model')

            if best_loss >= loss_test:
                best_loss = loss_test
                if best_loss < 5:
                    torch.save(model.state_dict(), '../log4/model_OMv7_fold{}_loss0208.pth'.format(i_fold))
                    print('saving model')

            print('Now fold={}, best_acc={}， best_auc={}, best_loss={}'
                  .format(i_fold, best_acc, best_auc, best_loss) )

            if auc_train >=0.9 and aug < 0.8:
                aug += 0.5
                train_dataset = M1_dataset(path_list=train_filepath_list,
                                   label_list=train_label_list, mode='train', aug=aug)
                train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

            if auc_train > 0.995:
                break
        
        T1 = time.time()
        log = 'Exp on 20230208 OnlyMaskResnet ------------------------------------------------------\n'
        log += 'Fold: %d, loss: %.4f, acc: %.4f, auc: %.4f, time: %s\n' % (
            i_fold, best_loss, best_acc, best_auc, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
        log += 'saved_gate: {}\n'.format(best_gate)
        f = open('../log4/log_OM.txt', 'a')
        f.write(log)
        f.close()
        
np.save('trainig_history_mask.npy', history) # 注意带上后缀名


Building dataloader: training: 746, testing: 83



/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


| Epoch [  0/100] Iter[ 10/ 47]	 CE-loss: 0.7036
| Epoch [  0/100] Iter[ 20/ 47]	 CE-loss: 0.7116
| Epoch [  0/100] Iter[ 30/ 47]	 CE-loss: 0.7428
| Epoch [  0/100] Iter[ 40/ 47]	 CE-loss: 0.7268
| Epoch [  0/100] Iter[ 47/ 47]	 CE-loss: 0.6675
Now evaluating on fold 0

| Test Epoch #0	 Accuracy: 57.10%	 Loss: 0.04	 AUC: 0.753	 SPE: 0.738


| Test Epoch #0	 Accuracy: 57.83%	 Loss: 0.05	 AUC: 0.773	 SPE: 0.770

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], device='cuda:0')
saving model
Now fold=0, best_acc=57.83132530120482， best_auc=0.7726190476190475, best_loss=0.04798396021486765
| Epoch [  1/100] Iter[ 10/ 47]	 CE-loss: 0.7276
| Epoch [  1/100] Iter[ 20/ 47]	 CE-loss: 0.5258
| Epoch [  1/100] Iter[ 30/ 47]	 CE-loss: 0.6341
| Epoch [  1/100] Iter[ 40/ 47]	 CE-loss: 0.5270
| Epoch [  1/100] Iter[ 47/ 47]	 CE-loss: 0.6543
Now evaluating on fold 0

| Test Epoch #1	 Accuracy: 76.54%	 Loss: 0.03	 AUC: 0.848	 SPE: 0.848


| Test Epoch #1	 Accuracy: 75.90%	 Loss: 0.04	 AUC: 0.838	 SPE: 0

| Epoch [ 14/100] Iter[ 10/ 47]	 CE-loss: 0.2029
| Epoch [ 14/100] Iter[ 20/ 47]	 CE-loss: 0.3359
| Epoch [ 14/100] Iter[ 30/ 47]	 CE-loss: 0.2573
| Epoch [ 14/100] Iter[ 40/ 47]	 CE-loss: 0.5053
| Epoch [ 14/100] Iter[ 47/ 47]	 CE-loss: 0.1822
Now evaluating on fold 0

| Test Epoch #14	 Accuracy: 90.75%	 Loss: 0.02	 AUC: 0.965	 SPE: 0.967


| Test Epoch #14	 Accuracy: 81.93%	 Loss: 0.03	 AUC: 0.884	 SPE: 0.886

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000], device='cuda:0')
Now fold=0, best_acc=84.33734939759036， best_auc=0.9107142857142857, best_loss=0.029147323983979512
| Epoch [ 15/100] Iter[ 10/ 47]	 CE-loss: 0.2305
| Epoch [ 15/100] Iter[ 20/ 47]	 CE-loss: 0.4580
| Epoch [ 15/100] Iter[ 30/ 47]	 CE-loss: 0.2900
| Epoch [ 15/100] Iter[ 40/ 47]	 CE-loss: 0.4794
| Epoch [ 15/100] Iter[ 47/ 47]	 CE-loss: 0.2624
Now evaluating on fold 0

| Test Epoch #15	 Accuracy: 92.63%	 Loss: 0.01	 AUC: 0.977	 SPE: 0.978


| Test Epoch #15	 Accuracy: 89.16%	 Loss: 0.03	 AUC: 0.919	 SPE: 0.923

te

KeyboardInterrupt: 